In [1]:
%matplotlib notebook
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

import ipywidgets as widgets
from IPython.display import display
from fbprophet import Prophet

Duplicate key in file 'C:\\Users\\User\\.matplotlib\\stylelib\\mystyle_med.mplstyle' line #40.
Duplicate key in file 'C:\\Users\\User\\.matplotlib\\stylelib\\mystyle_med.mplstyle' line #41.
Duplicate key in file 'C:\\Users\\User\\.matplotlib\\stylelib\\mystyle_sw.mplstyle' line #40.
Duplicate key in file 'C:\\Users\\User\\.matplotlib\\stylelib\\mystyle_sw.mplstyle' line #41.


ModuleNotFoundError: No module named 'fbprophet'

In [ ]:
def getTimeSeries(fname):
    df = pd.read_csv(fname)
    countryData = df.groupby('Country/Region').sum()
    countryData = countryData.drop(['Lat', 'Long'], axis=1).T
    return countryData.set_index(pd.DatetimeIndex(countryData.index))

In [ ]:
def fetchCountryData(country, freq='1d'):
    confirmed = getTimeSeries('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv')
    dead = getTimeSeries('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv')
    recovered = getTimeSeries('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv')

    data = pd.DataFrame([confirmed[country], dead[country], recovered[country]],
                ['confirmed', 'dead', 'recovered']).T
    
    data = data[data.confirmed>0].resample(freq).max() # start with first confirmed case
    data[['new_confirmed', 'new_dead', 'new_recovered']] = data.diff()
    
    data['mortality'] = data.dead / data.confirmed * 100
    
    return data

In [ ]:
def fetchCountryList():
    df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv')
    countryData = df.groupby('Country/Region').sum()
    return sorted(df['Country/Region'].unique())


In [ ]:
def plotAbsoluteCases(data):

    fig, ax = plt.subplots(nrows=2, sharex=True, figsize=(8,8))
    
    ax[0].set_title('confirmed (%d)' %data.confirmed[-1])
    data.confirmed.plot(kind='bar', ax=ax[0])

    ax[1].set_title('dead (%d) and recovered (%d)' %(data.dead[-1], data.recovered[-1]))
    data[['dead','recovered']].plot(kind='bar', ax=ax[1], color='rgb', rot=45)
    
    ax[1].xaxis.set_major_formatter(plt.FixedFormatter(data.index.to_series().dt.strftime("%b %d")))
    
    for axis in ax:
        axis.grid()

In [ ]:
def plotPredictions(confirmed):
    prophet_basic = Prophet ()
    prophet_basic.fit (confirmed)

    # making data frame with the future values
    future = prophet_basic.make_future_dataframe (periods=300)  # periods - number of the datas for which to
    future.tail (2)
    forecast = prophet_basic.predict (future)

    fig1 = prophet_basic.plot (forecast)


In [ ]:
def plotNewCases(data):

    fig, ax = plt.subplots(nrows=2, sharex=True, figsize=(8,8))

    ax[0].set_title('new confirmed (%d)' %data.new_confirmed[-1])
    data.new_confirmed.plot(kind='bar', ax=ax[0])

    ax[1].set_title('new dead (%d) and recovered (%d)' %(data.new_dead[-1], data.new_recovered[-1]))
    data[['new_dead','new_recovered']].plot(kind='bar', ax=ax[1], color='rg', rot=45)

    ax[1].xaxis.set_major_formatter(plt.FixedFormatter(data.index.to_series().dt.strftime("%b %d")))
    for axis in ax:
        axis.grid()  

In [ ]:
output_stats = widgets.Output()
output_plot_absoluteCases = widgets.Output()
output_plot_predictedCases = widgets.Output()
output_plot_newCases = widgets.Output()
output_mortality = widgets.Output()

button_refresh = widgets.Button(description = 'Refresh')
def on_refresh(_):
    common_filtering(dropdown_countries.value, dropdown_frequencies.value)

button_refresh.on_click(on_refresh)
    
    
dropdown_countries = widgets.Dropdown(options = fetchCountryList(),
                                     description = 'Country:')

frequencies = ('d', 'w', 'M', 'Q', '3Q')
dropdown_frequencies = widgets.Dropdown(options = frequencies,
                                      description = 'Frequency:')

text_mortality = widgets.Text(disabled = True,
                             description = 'Mortality:',
                             value = "NaN")

def dropdown_countries_eventhandler(change):
    common_filtering(change.new, dropdown_frequencies.value)

def dropdown_frequencies_eventhandler(change):
    common_filtering(dropdown_countries.value, change.new)    

output = widgets.Output()
def common_filtering(country, freq):
    output_stats.clear_output()
    output_plot_absoluteCases.clear_output()
    output_plot_newCases.clear_output()
    output_plot_predictedCases.clear_output()

    with output_stats:
        data = fetchCountryData(country, freq)
        display(data)
    with output_plot_absoluteCases:
        display(plotAbsoluteCases(data))
    with output_plot_newCases:
        display(plotNewCases(data))
    with output_mortality:
        text_mortality.value = "{:.2f}%".format(data.mortality[-1])
    with output_plot_predictedCases:
        display (plotPredictions (data.confirmed))


dropdown_countries.observe(dropdown_countries_eventhandler, names='value')
dropdown_frequencies.observe(dropdown_frequencies_eventhandler, names='value')

In [ ]:
input_widgets = widgets.HBox([dropdown_countries, dropdown_frequencies, button_refresh])

tab = widgets.Tab([output_plot_absoluteCases, output_plot_newCases, output_stats, output_plot_predictedCases])
tab.set_title(0, 'Total Cases')
tab.set_title(1, 'New Cases')
tab.set_title(2, 'Data Set')
tab.set_title(3, 'Predictions')

dashboard = widgets.VBox((input_widgets, text_mortality, tab))

In [ ]:
display(dashboard)
button_refresh.click()